Inserire il path all'interno del quale è contenuto il file .ipynb

In [ ]:
path = "/content/drive/MyDrive/CODICE FINALE/"

Unzip del database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Inserire il path all'interno del quale è contenuto il construction set

In [1]:
!unzip "/content/drive/MyDrive/OCTchallenge_new.zip" # inserire path completa in cui si trovano i dati

unzip:  cannot find or open /content/drive/MyDrive/OCTchallenge_new.zip, /content/drive/MyDrive/OCTchallenge_new.zip.zip or /content/drive/MyDrive/OCTchallenge_new.zip.ZIP.


# Installazione di MONAI ed esportazione delle librerie

Istallazione di MONAI

In [ ]:
!pip install 'monai[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 KB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━

Esportazione delle librerie

In [ ]:
!pip install pynrrd

import os
from monai.inferers import SimpleInferer
import nrrd

from monai.utils import set_determinism
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.config import print_config
from monai.metrics import DiceMetric,ConfusionMatrixMetric

import monai
import matplotlib.pyplot as plt
import numpy as np
import torch 
import PIL
from PIL import ImageEnhance

from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize

from skimage.transform import rotate
from scipy.ndimage import binary_fill_holes, median_filter, gaussian_filter,binary_dilation, binary_erosion
from skimage.measure import label, regionprops, regionprops_table
import matplotlib.patches as mpatches

import skimage.io as io
from skimage import morphology,exposure
import cv2
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image, ImageFilter, ImageEnhance

from monai.transforms import (
    AsDiscrete,
    DataStatsd,
    AddChanneld,
    Compose,
    Activations,
    LoadImage,
    LoadImaged,
    Resize,
    Resized,
    RandFlipd,
    ScaleIntensityRange,
    ScaleIntensityRanged,
    DataStats,
    AsChannelFirstd,
    AsDiscreted,
    ToTensord,
    EnsureType,
    ThresholdIntensityd,
    SpatialCropd,
    CropForegroundd,
    EnsureChannelFirstd,
    RandSpatialCropSamplesd,
    Orientationd,
    Spacingd,
    DataStatsd,
    RandRotate90d,
    RandRotated,
    RandAxisFlipd,
    AdjustContrastd,
)

from monai.data import (
    DataLoader,
    CacheDataset,
    PILReader,
    ITKReader,
    NrrdReader,
    IterableDataset,
    decollate_batch,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Definizione dei path

In [ ]:
current_dir = os.getcwd()
dataset_name = 'new dataset' #nome della cartella in cui è contenuto il construction set

#Poichè all'interno delle cartelle di training e validation vi sono due sotto-cartelle (w6_d5 e w6_d7), 
#sono state create rispettivamente due liste ordinate (sia per le immagini che per le maschere) per training e validation.

training1_volumes_path = os.path.join(dataset_name,'Volumes','training','w6_d5')
training2_volumes_path = os.path.join(dataset_name,'Volumes','training','w6_d7')

validation1_volumes_path = os.path.join(dataset_name,'Volumes','validation','w2_d7')
validation2_volumes_path = os.path.join(dataset_name,'Volumes','validation','w6_d9')

training1_masks_path = os.path.join(dataset_name,'masks','training','w6_d5')
training2_masks_path = os.path.join(dataset_name,'masks','training','w6_d7')

validation1_masks_path = os.path.join(dataset_name,'masks','validation','w2_d7')
validation2_masks_path = os.path.join(dataset_name,'masks','validation','w6_d9')

#liste ordinate per le immagini di training --> 4608 elementi
training1_volumes = sorted(os.listdir(training1_volumes_path))
training1_volumes = training1_volumes[1:] #al fine di eliminare il primo elemento che è ".DS_Store", ovvero un file nascosto con un formato proprietario creato da macOS 
training2_volumes = sorted(os.listdir(training2_volumes_path))

#liste ordinate per le maschere di training
training1_masks = sorted(os.listdir(training1_masks_path))
training2_masks = sorted(os.listdir(training2_masks_path))

#liste ordinate per le immagini di validazione --> 1024 elementi 
validation1_volumes = sorted(os.listdir(validation1_volumes_path))
validation2_volumes = sorted(os.listdir(validation2_volumes_path))

#liste ordinate per le maschere di validazione
validation1_masks = sorted(os.listdir(validation1_masks_path))
validation2_masks = sorted(os.listdir(validation2_masks_path))

# Creazione del training e validation set dict

Creazione del training set dict

In [ ]:
training_data = [] # Inizializzazione della lista di dizionari del training set
for i in range(len(training1_volumes)): #ciclo sul primo set di immagini di training 
    tempDict = {
        'image': os.path.join(training1_volumes_path, training1_volumes[i]),
        'segmentation': os.path.join(training1_masks_path, training1_masks[i])}
    
    training_data.append(tempDict)

for i in range(len(training2_volumes)): #ciclo sul secondo set di immagini di training

    tempDict = {
        'image': os.path.join(training2_volumes_path, training2_volumes[i]),
        'segmentation': os.path.join(training2_masks_path,training2_masks[i])}
    
    training_data.append(tempDict)

Creazione del validation set dict

In [ ]:
validation_data = [] # Inizializzazione della lista di dizionari del validation set
for i in range(len(validation1_volumes)): #ciclo sul primo set di immagini di validazione

    tempDict = {
        'image': os.path.join(validation1_volumes_path, validation1_volumes[i]),
        'segmentation': os.path.join(validation1_masks_path, validation1_masks[i])}
    
    validation_data.append(tempDict)

for i in range(len(validation2_volumes)): #ciclo sul secondo set di immagini di validazione
    tempDict = {
        'image': os.path.join(validation2_volumes_path, validation2_volumes[i]),
        'segmentation': os.path.join(validation2_masks_path,validation2_masks[i])}
    
    validation_data.append(tempDict)

# Pre- e Post- processing

Funzione che permette di generare una maschera binaria, tramite la tecnica del global thresholding, a partire dall'immagine pre elaborata.

*   x: immagine 2D pre-processata
*   Threshold: soglia per il global thresholding

In [ ]:
def img_to_mask(x,threshold): 
  mask= x.copy()
  mask[mask<threshold] = 0 
  mask[mask>threshold] = 1
  return mask

Funzione che permette di gestire gli artefatti ti dipo lineare

*   x: immagine 2D pre-processata
*   mask: maschera ottenuta dalla funzione img_to_mask



In [ ]:
def delartifact(x,mask):
  kernel = np.ones((3,3)) #Definizione dell'elemento strutturale (kernel 3x3)
  dilated = binary_dilation(mask, structure=kernel) # Dilatazione degli elementi con intensità di pixel=1
  mask= binary_fill_holes(mask) 
  mask = binary_erosion(dilated,structure=kernel) # Erosione degli elementi con intensità di pixel=1

  image_mask= x*mask #Maschera sovrapposta all'immagine
  regions = regionprops(label(mask),image_mask) # Calcola le proprietà delle regioni

  for region in regions:
    tumor_coords = region.coords # Estrazione delle coordinate dei pixel all'interno della regione del tumore      
    minr, minc, maxr, maxc = region.bbox #Estrazione delle coordinate della bounding box

    if ((maxr-minr)>50 and (maxc-minc)<35) or ((maxr-minr)>150 and (maxc-minc)<37) or ((maxr-minr)<37 and (maxc-minc)>150) or ((maxr-minr)>20 and (maxc-minc)<8): #Condizione per eliminare l'artefatto
      x[tumor_coords[:, 0], tumor_coords[:, 1]]=np.min(x) #eliminazione dell'artefatto dall'immagine

  return x

Definizione di una funzione di pre processing


*   x: immagine 2D originale
*   y: maschera manuale associata all'immagine x



In [ ]:
def preprocessing(x,y):
  
  #Eliminazione del canale --> (256,256)
  x = x.squeeze()
  y = y.squeeze()

  #Conversione delle immagini in formato numpy
  x = x.cpu().numpy()
  y = y.cpu().numpy()
  
  #Applicazione del filtro mediano
  x = median_filter(x,3)

  #Gestione degli artefatti
  mask = img_to_mask(x,1700)
  condizione = np.count_nonzero(mask)
  if condizione !=0: #se la maschera ha almeno un elemento uguale a 1
    x = delartifact(x,mask)
  
  return x,y

Definizione di una funzione di post processing

*   x: maschera automatica in output al modello 
*   y: maschera manuale associata all'immagine z
*   z: immagine 2D pre-processata 



In [ ]:
def postprocessing(x,y,z):
  
  #Conversione delle immagini in formato numpy
  x = x.cpu().numpy()
  y = y.cpu().numpy()
  z = z.cpu().numpy()

  #Riempimento dei buchi della maschera automatica
  x = binary_fill_holes(x)

  return x,y,z

# Creazione dei Dataset e DataLoader per le fasi di Training e Validation

In [ ]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image","segmentation"],image_only=False, reader=ITKReader()), # caricamento dell'immagine e della segmentazione
        EnsureChannelFirstd(keys=["image","segmentation"]), # aggiunge il canale all' immagine e alla segmentazione ottenendo il formato channel-first da (NxN) a (1xNxN)
        Resized(keys=["image", "segmentation"], spatial_size=[256,256]), # resize dell'immagine a 256x256
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5
        AdjustContrastd(keys=["image"],gamma=1.5),
     
        # Operazioni di data augmentation: 
        RandRotated(keys=["image", "segmentation"], range_x=[-0.1, 0.1], prob=0.3, padding_mode="zeros"),
    
        #DataStatsd(keys = ['image', 'segmentation']), #Per verificare le dimensioni finali dell'immagine
        ToTensord(keys=["image", "segmentation"]), # per portare immagini e segmentazioni da formato PIL a un torch tensor da dare in input alla rete
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["image","segmentation"],image_only=False, reader=ITKReader()), # caricamento dell'immagine e della segmentazione
        EnsureChannelFirstd(keys=["image","segmentation"]), # aggiunge il canale all' immagine e alla segmentazione ottenendo il formato channel-first da (NxN) a (1xNxN)
        Resized(keys=["image", "segmentation"], spatial_size=[256,256]), # resize dell'immagine a 256x256
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        AdjustContrastd(keys=["image"],gamma=1.5),
        ToTensord(keys=["image", "segmentation"]), # per portare immagini e segmentazioni da formato PIL a un torch tensor da dare in input alla rete
    ]
)

In [ ]:
batch_size= 32

#Creazione di un oggetto iterabile da cui è possibile estrarre in maniera sequenziale dei dati
#In input vengono fornite le coppie immagine-segmentazione e le trasformazioni da applicare
train_ds = IterableDataset(data = training_data, transform = train_transforms)

wks = 0 # definisce il numero di workers (core della CPU da utilizzare) per caricare le immagini

# definizione dell'oggetto Dataloader che si occuperà di caricare le immagini in maniera sequenziale
# definendo i vari batch durante l'allenamento
train_loader = DataLoader(train_ds, batch_size=batch_size, num_workers=wks, pin_memory=True, shuffle=False)

#Le stesse operazioni vengono ripetute per il validation
val_ds = IterableDataset(data = validation_data, transform = val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=wks, pin_memory=True, shuffle=False)

# Creazione della rete

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Definizione del device da utilizzare: GPU

model = UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH, #feature normalization type and arguments (Defaults to instance norm)
).to(device)

# Definizione della loss function
from monai.losses.dice import DiceCELoss
loss_function = DiceCELoss(sigmoid = True) 

# Definizione dell'ottimizatore
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Definizione delle trasformazioni di post-processing
post_pred = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)]) 
post_label = Compose([EnsureType(), AsDiscrete(threshold=0.5)])
post_image = Compose([EnsureType()])

# Definizione del tipo di Inferenza da applicare con il modello allenato
inferer = SimpleInferer()

# Definizione un seed per renderere ripetibili tutte le opearazioni che andiamo ad effettuare
# Questa operazione non è sempre possibile in quanto alcune operazioni hanno condizioni aleatorie intrinseche,
# in quel caso possono essere segnalati errori e warning che risolviamo eliminando questa condizione
set_determinism(seed=46)

**Creazione cartella** per salvataggio dei risultati relativi all'allenamento della rete

In [ ]:
expFd = os.path.join(path,'Validation-Snapshot')

if not os.path.isdir(expFd):
  os.mkdir(expFd) #crea la directory

FileNotFoundError: ignored

# SALVATAGGIO IMMAGINI PRE-PROCESSING  -*NON ESEGUIRE*-

**Creazione cartella** per salvataggio dei risultati relativi alla fase di pre-processing

In [ ]:
expFd2 = os.path.join(path,'Pre-processing')

if not os.path.isdir(expFd2):
  os.mkdir(expFd2) #crea la directory

Funzione per il salvataggio delle immagini in formato .nrrd

*   list_val_output: lista di tensori con shape [1,1,256,256]
*   stepFd: path della cartella all'interno della quale salvare le immagini
*   fdName: path dell'immagine


In [ ]:
def Savenrrd(list_val_output,stepFd,fdName):
  
  #Estrazione del nome dell'immagine a partire da fdName
  parts = fdName.split('/') # suddivide la stringa in base al separatore '/'
  parts = parts[4:] # elimina la prima parte della lista
  string = '/'.join(parts) # ricostruisci la stringa eliminando la parte specificata

  file_path = os.path.join(stepFd, '{}.nrrd'.format(string[:-4] + '_intera')) # crea il percorso completo del file in cui salvare l'immagine

  image_3d = torch.cat(list_val_output, dim=1) # Concatena le slice lungo la dimensione dei canali
  image_3d_numpy = image_3d.squeeze().cpu().numpy().transpose(1,2,0) # trasforma image_3d in un array numpy - image_3d_numpy.shape --> (256,256,32)
  
  nrrd.write(file_path, image_3d_numpy) # scrivi l'array NumPy in un file .nrrd

Codice per salvare le immagini volumetriche **dopo l'applicazione del pre-processing**

In [ ]:
eval_num = int(len(training_data)/batch_size) # definire numero di iterazioni per effettuare validazione ogni epoca 
max_iterations = eval_num # numero massimo di batch da iterare
global_step = 1 

x_3D= () #Inizializzazione della lista che conterrà le 256 slices che compongono il volume dell'immagine pre-processate
y_3D=() #Inizializzazione della lista che conterrà le 256 slices che compongono il volume delle maschere segmentate dall'operatore

while global_step < max_iterations: # loop per richiamare la funzione train
  step = 0
  epoch_iterator = tqdm(train_loader, desc="Pre-processing (X / X Steps)", dynamic_ncols=True, total=eval_num, position=0, leave=True) # crea un oggetto tqdm per tracciare l'avenzamento delle epoche

  for step, batch in enumerate(epoch_iterator): # ad ogni ciclo for estrae una coppia step (numero dell'iterazione), batch (immagine e segmentazione) dal train loader
      step += 1

      x1, y1 = (batch["image"], batch["segmentation"]) # manda i tensori di immagine e maschera alla GPU --> (32,1,256,256)

      for i in range(batch_size):
        x2,y2 = x1[i,:,:,:], y1[i,:,:,:]  #Estrazione del primo elemento del batch --> Metatensor (1,256,256)

        #Applicazione della funzione pre-processing
        x2,y2 = preprocessing(x2,y2) #--> x2/y2.size (256,256)
        
        Totensor = transforms.ToTensor() # permette di trasformare array numpy in tensori (Questa operazione ruota l'immagine)
        if i==0: # prima iterazione
          x = Totensor(x2)
          x = torch.unsqueeze(x,0).float() # aggiungo un canale

          y = Totensor(y2)
          y = torch.unsqueeze(y,0)
        else: # successive iterazione
          x3 = Totensor(x2)
          x3 = torch.unsqueeze(x3,0).float()
          x = torch.row_stack((x,x3))

          y3 = Totensor(y2)
          y3 = torch.unsqueeze(y3,0)
          y = torch.row_stack((y,y3))

      #forma che ci aspettiamo --> [batch_size, channels, height, width] - x/y.shape --> (32,1,256,256)

      tupla_x= torch.split(x, 1, dim=0) #dividere il tensore x in 32 tensori con shape [1,1,256,256]
      x_3D = x_3D + tupla_x #aggiunge la lista "tupla_x" alla lista x_3D

      tupla_y= torch.split(y, 1, dim=0)
      y_3D = y_3D + tupla_x

      if len(x_3D)==256: #Condizione per il salvataggio del volume 3D
        fdName = batch['image_meta_dict']['filename_or_obj'][-1][:-4]
        Savenrrd(x_3D,expFd2,fdName)
        x_3D=()
        y_3D=()

      epoch_iterator.set_description("Pre-processing (%d / %d Steps)" % (global_step, max_iterations)) # aggiorna il counter tqdm

      global_step += 1

# Funzione di **training**

Funzione per il salvataggio delle immagini in formato .nrrd

*   list_val_output: lista di tensori con shape [1,1,256,256]
*   stepFd: path della cartella all'interno della quale salvare le immagini
*   fdName: path dell'immagine


In [ ]:
def Savenrrd(list_val_output,stepFd,fdName,intera):
  
  #Estrazione del nome dell'immagine a partire da fdName
  parts = fdName.split('/') # suddividi la stringa in base al separatore '/'
  parts = parts[4:] # elimina la prima parte della lista
  string = '/'.join(parts) # ricostruisci la stringa eliminando la parte specificata

  file_path = os.path.join(stepFd, '{}.nrrd'.format(string[:-4] + '_output')) # crea il percorso completo del file in cui salvare l'immagine

  image_3d_numpy = list_val_output.squeeze().cpu().numpy().transpose(1,2,0) # trasforma image_3d in un array numpy - image_3d_numpy.shape --> (256,256,32)
  
  nrrd.write(file_path, image_3d_numpy) # scrive l'array NumPy in un file .nrrd

Funzione che permette di trasformare le matrici (256,256) numpy in tensori con dimensioni [batch_size, channels, height, width]

*   x2: immagine 2D pre-processata (formato numpy) 
*   y2: maschera manuale 2D (formato numpy) 
*   i: indica l'i-esima immagine del batch
*   d: per indicare se si tratta di un operazione che viene svolta nel processo di pre o post-processing
*   tipo: per indicare se le immagini/maschere in input sono tensori o meno
*   x= tensore di immagini creato nell'iterazione precedente nel formato nel formato [batch_size, channels, height, width]
*   y= tensore di maschere creato nell'iterazione precedente nel formato [batch_size, channels, height, width]





In [ ]:
def NumTensor(x2, y2, i, d,tipo, x=None, y=None):
  Totensor = transforms.ToTensor() # permette di trasformare array numpy in tensori (Questa operazione ruota l'immagine)

  if d == 'pre': #operazioni che vengono applicate sulle immagini di pre-processing
  #---------  
    if tipo == 'N-3D': #operazioni che vengono eseguite quando x2 e y2 non sono tensore
      x3 = Totensor(x2).unsqueeze(0).float().to('cuda') #si utilizza la gpu
      y3 = Totensor(y2).unsqueeze(0).to('cuda')
    else: #operazioni che vengono eseguite quando x2 e y2 sono già tensore
      x3=x2
      y3=y2

    if i == 0: # Prima iterazione
      x, y = x3, y3
    else: # Iterazioni successive
      x = torch.cat((x, x3), dim=0) #concatena i tensori x3 a x
      y = torch.cat((y, y3), dim=0)
  #---------  
  else: #operazioni che vengono applicate sulle immagini di post-processing

    if tipo == 'N-3D': #operazioni che vengono eseguite quando x2 e y2 non sono tensore
      x3 = Totensor(x2).unsqueeze(0).float().to('cpu') #si utlizza la cpu
      y3 = Totensor(y2).unsqueeze(0).to('cpu')
    else: #operazioni che vengono eseguite quando x2 e y2 sono già tensore
      x3=x2
      y3=y2

    if i == 0: # Prima iterazione
      x, y = x3, y3
    else: # Iterazioni successive
      x = torch.cat((x, x3), dim=0) #concatena i tensori x3 a x
      y = torch.cat((y, y3), dim=0)
  return x, y

In [ ]:
def train(global_step, train_loader, dice_val_3D_best,recall_val_3D_best,precision_val_3D_best, global_step_best):

  model.train() # setta il modello in modalità training
  epoch_loss = 0
  step = 0
  x = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le immagini in output alla funzione di pre-processing
  y = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere manuali in output alla funzione di pre-processing

  epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True,total=eval_num, position=0, leave=True) # crea un oggetto tqdm per tracciare l'avenzamento delle epoche
  
  for step, batch in enumerate(epoch_iterator): # ad ogni ciclo for estrae una coppia step (numero dell'iterazione), batch (immagine e segmentazione) dal train loader
    step += 1

    x1, y1 = (batch["image"].cuda(), batch["segmentation"].cuda()) # manda i tensori di immagine e maschera alla GPU --> (32,1,256,256)

    for i in range(batch_size):
      x2,y2 = x1[i,:,:,:], y1[i,:,:,:]  #prendo il primo elemento del batch --> (1,256,256)

      #Applicazione della funzione pre-processing
      x2,y2 = preprocessing(x2,y2) #--> x2/y2.size (256,256)

      #Applicazione della funzione NumTensor
      x,y = NumTensor(x2, y2, i,d='pre',tipo='N-3D', x=x, y=y) # --> x/y.shape (32,1,256,256) [batch_size, channels, height, width]

    optimizer.zero_grad() # resetta i gradienti dell'optimizer
    logit_map = model(x) #l'immagine x viene data in input al modello
    loss = loss_function(logit_map, y) # calcolo della loss
    loss.backward() # effettua la backpropagation della loss
    epoch_loss += loss.item() # accumula i valori delle loss nei vari batch
    optimizer.step() # calcola i gradienti e aggiorna l'optimizer

    epoch_iterator.set_description("Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)) # aggiorna il counter tqdm

    if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations: # condizione per effettuare la validazione
        GB = global_step

        dice_val_3D,recall_val_3D,precision_val_3D = validation(GB) # richiama la funzione di validazione per estrarre le metriche da valutare sul validation set

        epoch_loss /= step # divide la loss in base al numero del batch
        epoch_loss_values.append(epoch_loss) # salva i valori della loss

        metric_values_dice_3D.append(dice_val_3D) # salva i valori della metrica dice 3D
        metric_values_precision_3D.append(precision_val_3D) # salva i valori della metrica precision 3D
        metric_values_recall_3D.append(recall_val_3D) # salva i valori della metrica recall 3D
        
        if dice_val_3D > dice_val_3D_best: # salvataggio del modello se la metrica di validazione è migliorata
          best_epoca= int(global_step/eval_num)
          dice_val_3D_best = dice_val_3D
          precision_val_3D_best = precision_val_3D
          recall_val_3D_best = recall_val_3D

          global_step_best = global_step
          torch.save( model.state_dict(), os.path.join(path, "best_metric_model.pth"))
          
          print("\nModello salvato! - Immagini 3D: Miglior Dice medio: {} (Epoca {}) - Miglior Precision media: {} - Miglior Recall media: {} - Attuale Dice medio: {} (Epoca {})\n".format(dice_val_3D_best,best_epoca,precision_val_3D_best,recall_val_3D_best, dice_val_3D,int(global_step/eval_num)))
        else:
          print("\nModello non salvato! - Immagini 3D: Miglior Dice medio: {}.       - Miglior Precision media: {} - Miglior Recall media: {} - Attuale Dice medio: {} (Epoca {})\n".format(dice_val_3D_best,precision_val_3D_best,recall_val_3D_best, dice_val_3D,int(global_step/eval_num)))
            
    global_step += 1
             
  return global_step, dice_val_3D_best,recall_val_3D_best,precision_val_3D_best, global_step_best

# Funzione di **validation**

In [ ]:
def validation(GB):
  model.eval() # setta il modello in moadlità inference
  
  epoch_val_loss = 0
  dice_3D= 0 #Inizializzazione del coefficiente di Dice 3D
  contatore_3D=0 #Inizializzazione di un contatore che possa aiutare durante la creazione dei metatensori per i volumi 3D

  dice_vals_3D = list() #Inizializzazione della lista che conterrà i valori di dice per ogni volume 3D
  precision_vals_3D = list() #Inizializzazione della lista che conterrà i valori di precision per ogni volume 3D
  recall_vals_3D = list() #Inizializzazione della lista che conterrà i valori di recall per ogni volume 3D

  val_inputs = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le immagini in output alla funzione di pre-processing
  val_labels4 = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere manuali in output alla funzione di pre-processing
  val_outputs = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere automatiche in output alla funzione di post-processing
  val_labels = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere manuali in output alla funzione di post-processing
  val_outputs_3D = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere automatiche per la generazione di un Metatensore nel formato [256,1,256,256] che contenga tutti i tensori di un volume 3D 
  val_labels_3D = torch.empty(32, 1, 256, 256, device='cuda') #inizializzazione di un tensore vuoto nel formato [32,1,256,256] per le maschere manuali per la generazione di un Metatensore nel formato [256,1,256,256] che contenga tutti i tensori di un volume 3D 

  epoch_iterator_val = tqdm(val_loader, desc="Validate (X Steps) (dice=X.X)", dynamic_ncols=True, total=int(len(validation_data)/batch_size) ,position=0, leave=True) 
          
  with torch.no_grad():
    for stepv, batch in enumerate(epoch_iterator_val):
      stepv += 1
      
      val_inputs1, val_labels1 = (batch["image"].to(device), batch["segmentation"].to(device)) # manda i tensori di immagine e maschera alla GPU --> (32,1,256,256)
      
      #PRE-PROCESSING
      for i in range(batch_size):
        val_inputs0,val_labels0 = val_inputs1[i,:,:,:], val_labels1[i,:,:,:] #val_inputs0/val_labels0.shape --> (1,256,256)

        #Applicazione della funzione pre-processing
        val_inputs2,val_labels5 = preprocessing(val_inputs0,val_labels0) # val_inputs2/val_labels5.size --> (256,256)

        #Applicazione della funzione NumTensor
        val_inputs,val_labels4 = NumTensor(val_inputs2, val_labels5, i,d='pre',tipo='N-3D', x=val_inputs, y=val_labels4) # val_inputs/val_labels4.shape --> (32,1,256,256) [batch_size, channels, height, width]
      
      val_outputs4 = inferer(val_inputs, model) # applicazione del modello all'input val_outputs4.shape--> (32,1,256,256)

      loss_val = loss_function(val_outputs4, val_labels4) # calcolo della loss
      epoch_val_loss += loss_val.item()

      val_outputs4 = [post_pred(i) for i in decollate_batch(val_outputs4.squeeze())] # scompone il tenosore nelle varie slice
      val_labels4  = [post_label(i) for i in decollate_batch(val_labels4.squeeze())] # lista di 32 item contenente un MetaTensor con shape [256,256]
      val_inputs  = [post_image(i) for i in decollate_batch(val_inputs.squeeze())]

      #POST-PROCESSING
      for i in range(batch_size):
        val_outputs2,val_labels2,val_inputs2 = val_outputs4[i], val_labels4[i],val_inputs[i] # tensori con shape [256,256]

        #Applicazione della funzione post-processing
        val_outputs2,val_labels2,val_inputs2 = postprocessing(val_outputs2,val_labels2, val_inputs2) #Post-processing

        #Applicazione della funzione NumTensor
        val_outputs,val_labels = NumTensor(val_outputs2, val_labels2, i,d='post',tipo='N-3D', x=val_outputs, y=val_labels) # val_outputs/val_labels.shape --> (32,1,256,256)

      #Applicazione della funzione NumTensor  
      val_outputs_3D,val_labels_3D = NumTensor(val_outputs, val_labels, contatore_3D,d='post',tipo='3D', x=val_outputs_3D, y=val_labels_3D)
      contatore_3D += 1

      if val_outputs_3D.shape[0] == 256: # Condizione per l'azzeramento del contatore_3D
        contatore_3D=0
        
        if dice_val_3D_best>=0.65: #Condizione per il salvataggio delle del volume 3D
          stepFd = os.path.join(expFd, 'Epoca ' + str(int(GB/eval_num))) #Creazione della cartella che conterrà le immagini i volumi 3D per una specifica epoca
          if not os.path.isdir(stepFd):
            os.mkdir(stepFd)

          fdName = batch['image_meta_dict']['filename_or_obj'][-1][:-4] #path dell'immagine 
          Savenrrd(val_outputs_3D,stepFd,fdName) #Applicazione della funzione Savenrrd per il salvataggio dei volumi
        
        #Calcolo del Dice di un intera immagine 3D
        dice_metric_3D(y_pred=val_outputs_3D, y=val_labels_3D)
        dice_3D = dice_metric_3D.aggregate().item() #Valore di Dice su 256 slices
        dice_vals_3D.append(dice_3D)

        #Calcolo della precision di un intera immagine 3D
        precision_metric_3D(y_pred=val_outputs_3D, y=val_labels_3D)
        precision_3D = precision_metric_3D.aggregate() #Valore di Precision su 256 slices
        precision_3D = precision_3D[0].item()
        precision_vals_3D.append(precision_3D)

        #Calcolo della recall di un intera immagine 3D
        recall_metric_3D(y_pred=val_outputs_3D, y=val_labels_3D)
        recall_3D = recall_metric_3D.aggregate() #Valore di Recall su 256 slices
        recall_3D = recall_3D[0].item()
        recall_vals_3D.append(recall_3D)  

        val_outputs_3D.zero_()
        val_labels_3D.zero_()

      epoch_iterator_val.set_description("Validate (%d Steps) (dice 3D=%2.5f)" % (stepv, dice_3D))  

    dice_metric_3D.reset()
    recall_metric_3D.reset()
    precision_metric_3D.reset()
    epoch_val_loss /= stepv
    epoch_val_loss_values.append(epoch_val_loss)

  #Dice    
  mean_dice_val_3D = np.mean(dice_vals_3D) #media delle medie dei valori di dice di tutte e 8 le immagini di validation

  #Precision
  mean_precision_val_3D = np.mean(precision_vals_3D) #media delle medie dei valori di precision di tutte e 8 le immagini di validation
  
  #Recall
  mean_recall_val_3D = np.mean(recall_vals_3D) #media delle medie dei valori di recall di tutte e 8 le immagini di validation

  return mean_dice_val_3D,mean_recall_val_3D,mean_precision_val_3D

# **Training** della rete

**Inizializzazione dei parametri** per l'allenamento della rete

In [ ]:
eval_num = int(len(training_data)/batch_size) #numero di iterazioni per effettuare validazione ogni epoca 
max_iterations = 65*eval_num # numero massimo di batch da iterare
global_step = 1
best_epoca = 0

dice_metric_3D = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
recall_metric_3D = ConfusionMatrixMetric(include_background=True, metric_name='sensitivity', compute_sample=False, reduction="mean", get_not_nans=False) 
precision_metric_3D = ConfusionMatrixMetric(include_background=True, metric_name='precision', compute_sample=False, reduction="mean", get_not_nans=False)

dice_val_3D_best = 0.0
precision_val_3D_best = 0.0
recall_val_3D_best = 0.0
global_step_best = 0

epoch_loss_values = []
epoch_val_loss_values = []
metric_values_dice_3D = []
metric_values_precision_3D = []
metric_values_recall_3D = []

**Training della rete**

In [ ]:
from tqdm.notebook import tqdm

while global_step < max_iterations: # loop per richiamare la funzione train 
    global_step, dice_val_3D_best,recall_val_3D_best,precision_val_3D_best, global_step_best = train(
        global_step, train_loader, dice_val_3D_best,recall_val_3D_best,precision_val_3D_best, global_step_best)    


# Risultati DICE e LOSS

Caricamento di "best_metric_model"



In [ ]:
#model.load_state_dict(torch.load(os.path.join(current_dir, "best_metric_model.pth"))) # carichiamo il modello salvato

Plot loss e dice

In [ ]:
plt.figure("train", (20, 6))

#Plot Loss
plt.subplot(1, 4, 1)
plt.title("Iteration Average Loss")
x_ = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
y1 = epoch_val_loss_values
plt.xlabel("Iteration")
plt.plot(x_, y,label="train BCE")
plt.plot(x_,y1,label="val BCE")
plt.legend(loc="upper right")

#Plot Dice
plt.subplot(1, 4, 2)
plt.title("Val Mean Dice 3D")
x_ = [eval_num * (i + 1) for i in range(len(metric_values_dice_3D))]
y = metric_values_dice_3D
plt.xlabel("Iteration")
plt.plot(x_, y,color='orange')

#Plot Precision
plt.subplot(1, 4, 3)
plt.title("Val Mean Precision 3D")

x_p_3D = [eval_num * (i + 1) for i in range(len(metric_values_precision_3D))]
y_p_3D = metric_values_precision_3D
plt.xlabel("Iteration")
plt.plot(x_p_3D, y_p_3D,color='gray')

#Plot recall
plt.subplot(1, 4, 4)
plt.title("Val Mean Recall 3D")

x_p_3D = [eval_num * (i + 1) for i in range(len(metric_values_recall_3D))]
y_p_3D = metric_values_recall_3D
plt.xlabel("Iteration")
plt.plot(x_p_3D, y_p_3D,color='yellow')
plt.show()